In [23]:
import pandas as pd

In [24]:
df = pd.read_csv(r"train.csv")

In [25]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
#label 1 for real
#label 0 for fake

In [26]:
df.iloc[:3]

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1


In [27]:
df.isna()

,id,title,author,text,label
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
20795,False,False,False,False,False
20796,False,False,False,False,False
20797,False,False,False,False,False
20798,False,False,False,False,False


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [29]:
df["text"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 20800 entries, 0 to 20799
Series name: text
Non-Null Count  Dtype 
--------------  ----- 
20761 non-null  object
dtypes: object(1)
memory usage: 162.6+ KB


In [10]:
#lets clean the dataset for ML model

In [30]:
import re
from nltk.corpus import stopwords

In [31]:
"""
First we remove the stopwords from text
We see that some part of "text" column are float values, so we are typecasting everything into string
"""

stop_words = set(stopwords.words("english"))

def cleanTxt(tx):
    tx = str(tx)
    tx = tx.lower()
    ts = re.findall(r"\b\w+\b", tx)
    tl = [word for word in ts if word not in stop_words]
    ctx = " ".join(tl)
    return ctx

df["no_stopwords"] = df["text"].apply(cleanTxt)

In [32]:
df.head()

,id,title,author,text,label,no_stopwords
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,house dem aide even see comey letter jason cha...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,ever get feeling life circles roundabout rathe...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,truth might get fired october 29 2016 tension ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,videos 15 civilians killed single us airstrike...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,print iranian woman sentenced six years prison...


In [33]:
"""
Lets import the libraries to train the model
"""
import import_ipynb

In [34]:
from Word2Vector import Word2VectorTransformer

importing Jupyter notebook from Word2Vector.ipynb


In [35]:
"""
Import the ML models and metrics 
"""
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [36]:
#lets create the pipeline
pipeline = Pipeline(
    [
        ("Word2Vec", Word2VectorTransformer(vector_size=100, window=5, min_count=1, workers=4)),
        ("classifier", LogisticRegression(max_iter=1000))
    ]
)

In [37]:
X = df["no_stopwords"]
y = df["label"]

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('Word2Vec', Word2VectorTransformer()),
                ('classifier', LogisticRegression(max_iter=1000))])

In [48]:
prediction = pipeline.predict(X_test)

In [51]:
accuracy = accuracy_score(y_test, prediction)
print("Accuracy: ",accuracy)

Accuracy:  0.9117788461538462


In [53]:
cm = confusion_matrix(y_test, prediction)
cr = classification_report(y_test, prediction)
print("Confusion Matrix:\n",cm)
print("\nClassification report:\n",cr)

Confusion Matrix:
 [[1956  176]
 [ 191 1837]]

Classification report:
               precision    recall  f1-score   support

           0       0.91      0.92      0.91      2132
           1       0.91      0.91      0.91      2028

    accuracy                           0.91      4160
   macro avg       0.91      0.91      0.91      4160
weighted avg       0.91      0.91      0.91      4160



In [40]:
"""
Now lets test our model on some other dataset
"""
df_test = pd.read_csv(r"test.csv")

In [44]:
df_test.head(10)

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
5,20805,Trump is USA's antique hero. Clinton will be n...,NaN,Trump is USA's antique hero. Clinton will be n...
6,20806,Pelosi Calls for FBI Investigation to Find Out...,Pam Key,"Sunday on NBC’s “Meet the Press,” House Minori..."
7,20807,Weekly Featured Profile – Randy Shannon,Trevor Loudon,You are here: Home / *Articles of the Bound* /...
8,20808,Urban Population Booms Will Make Climate Chang...,NaN,Urban Population Booms Will Make Climate Chang...
9,20809,NaN,cognitive dissident,don't we have the receipt?


In [56]:
df_test["no_stopwords"] = df_test["text"].apply(cleanTxt)

In [57]:
df_test.head()

,id,title,author,text,no_stopwords
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",palo alto calif years scorning political proce...
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,russian warships ready strike terrorists near ...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,videos nodapl native american leaders vow stay...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",first succeed try different sport tim tebow he...
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,42 mins ago 1 views 0 comments 0 likes first t...


In [63]:
df_test.fillna("", inplace=True)

In [64]:
pipeline.predict(df_test["no_stopwords"])

array([0, 1, 1, ..., 0, 1, 0], dtype=int64)

In [65]:
df_test["predictions"] = pipeline.predict(df_test["no_stopwords"])

In [66]:
df_test.head(20)

,id,title,author,text,no_stopwords,predictions
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",palo alto calif years scorning political proce...,0
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,russian warships ready strike terrorists near ...,1
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,videos nodapl native american leaders vow stay...,1
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",first succeed try different sport tim tebow he...,0
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,42 mins ago 1 views 0 comments 0 likes first t...,1
5,20805,Trump is USA's antique hero. Clinton will be n...,,Trump is USA's antique hero. Clinton will be n...,trump usa antique hero clinton next president ...,1
6,20806,Pelosi Calls for FBI Investigation to Find Out...,Pam Key,"Sunday on NBC’s “Meet the Press,” House Minori...",sunday nbc meet press house minority leader re...,0
7,20807,Weekly Featured Profile – Randy Shannon,Trevor Loudon,You are here: Home / *Articles of the Bound* /...,home articles bound weekly featured profile ra...,1
8,20808,Urban Population Booms Will Make Climate Chang...,,Urban Population Booms Will Make Climate Chang...,urban population booms make climate change wor...,1
9,20809,,cognitive dissident,don't we have the receipt?,receipt,1
